In [1]:
import requests
from bs4 import BeautifulSoup

In [82]:
base_url = "https://rentahouse.com.ve/"

In [83]:
def get_links(base_url, page=1, casa=True):
    if casa:
        url = base_url + "casa.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Casa' }
    else:
        url = base_url + "apartamento.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Apartamento' }
    
    req = requests.get(url, params=payload)
    soup = BeautifulSoup(req.text, "html.parser")
    homes = soup.find_all("div", class_='property-list')

    res = []
    for home in homes:
        res.append(home.a['href'])
    
    return res

In [86]:
links = get_links(base_url)

In [129]:
def get_info_from_house(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    info = soup.find('div', class_="propertyInfo").find('div', class_="row")
    data = {}

    data['Price'] = info.find('div', class_='price')['title']

    general_info = info.find('ul', class_="property-detailes-list").find_all('li')

    for item in general_info:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    location = info.find_all('div', class_='DescripcionGeneral')[-1].find_all('li')

    for item in location:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    return data

In [130]:
get_info_from_house(links[0])

{'Price': '$40,000 USD',
 'Codígo RAH:': 'VE 23-26193',
 'Tipo de Propiedad:': 'Apartamento',
 'Estilo:': '1 Nivel',
 'Área Privada:': '92 m2\n\n',
 'Estado Del Inmueble:': 'Usado',
 'Dormitorios:': '2',
 'Total Baños:': '2',
 'Baños Completos:': '2',
 'Tipo De Estacionamiento:': 'Cubierto',
 'Puestos De Estacionamiento:': '2',
 'Amoblado:': 'Si',
 'País: ': 'Venezuela',
 'Estado: ': 'Lara',
 'Ciudad: ': 'Barquisimeto',
 'Urbanización: ': 'Avenida Libertador'}